In [1]:
# 1. scrape today's games with tipoff time (@ like 12pm)
# 2. create a custom yml to run 30 minutes before each of the games on this day
# 3. the script -- for all relevant games -- calls the odds api to fetch and save odds

In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import re
import calendar
from time import sleep

from helpers.scrape import get_first_basket
from helpers.utils import getId

In [5]:
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

months = list(calendar.month_name)[1:]

def get_monthly_games(month_url) :

    url = f'https://www.basketball-reference.com/{month_url}'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    table = soup.find('table')
    while table.find_all('tr', class_ = 'thead') :
        table.find('tr', class_ = 'thead').decompose()
    games = pd.read_html(str(table))[0].rename(columns = {'Start (ET)': 'Time'})
    games['Date'] = pd.to_datetime(games['Date'])
    games['Home'] = [x['href'].split('/')[2] for x in table.find_all('a', href = True) if 'teams' in x['href']][1::2]
    games['Away'] = [x['href'].split('/')[2] for x in table.find_all('a', href = True) if 'teams' in x['href']][0::2]
    games['game_id'] = [getId(x) for x in table.find_all('a', href = True) if 'boxscores' in x['href']][1::2]

    return games[['game_id', 'Date', 'Time', 'Home', 'Away']]


In [7]:
season = 2018
url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
page = requests.get(url)
if page.status_code == 429 :
    raise ValueError('Rate limited...')
soup = BeautifulSoup(page.content, 'lxml')
month_urls = [x['href'] for x in soup.find_all('a', href = True) if 'games' in x['href'] 
                and any(m.lower() in x['href'] for m in months)]

for m, month_url in enumerate(month_urls) :

    games_monthly = get_monthly_games(month_url)
    games_monthly['season'] = season

    break

        # first_basket_info = []
        # for i, gameId in enumerate(games_monthly['game_id'])  :

        #     print(f'[{round(100*(i+1)/len(games_monthly))}%...] season :  {season-1}-{season}, month :  {month_url.split("-")[-1].split(".")[0]} ({gameId})')
        #     sleep(5)
        #     first_basket_info.append(get_first_basket(gameId))


        # first_basket_df = pd.concat(first_basket_info)
        # first_basket_df = games_monthly.merge(first_basket_df, on = ['game_id', 'Home', 'Away'], how = 'inner')
        # if m == 0 :
        #     first_basket_df.to_csv(f'data/first_basket_{season}.csv', index = False)
        # else :
        #     first_basket_df.to_csv(f'data/first_basket_{season}.csv', mode = 'a', header = False, index = False)


SSLError: HTTPSConnectionPool(host='www.basketball-reference.com', port=443): Max retries exceeded with url: /leagues/NBA_2018_games.html (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1147)')))

In [4]:
url = 'https://www.rotowire.com/basketball/nba-lineups.php'

In [5]:
page = requests.get(url)

In [19]:
url = "https://www.rotowire.com/basketball/nba-lineups.php"
response = requests.get(url)
html = response.text.replace("<!--", "").replace(
    "-->", ""
)  # uncomment tables
soup = BeautifulSoup(html, "lxml")

In [20]:
len([x for x in soup.find_all('table')])

1

In [23]:
pd.read_html(str(table))[0]

,0,1,2
0,📱NBA Sportsbook,🎁 NBA Bonus,🤑 NBA Promo Code
1,📱BetMGM,"🎁 $1,500 First Bet Offer",🤑 ROTOBONUS
2,📱Caesars Sportsbook,"🎁 $1,000 First Bet on Caesars",🤑 ROTO1000
3,📱Fanatics Sportsbook,"🎁 Bet & Get $1,000 in No Sweat Bets",🤑 CLICK HERE
4,📱DraftKings,"🎁 Bet $5, Win $150 In Bonus Bets",🤑 CLICK HERE
5,📱FanDuel,"🎁 Bet $5, Win $150 in Bonus Bets + 3 Months of...",🤑 CLICK HERE
6,📱bet365,"🎁 $1,000 First Bet Safety Net or Bet $5, Get $150",🤑 ROTOWIRE


In [24]:
soup.find_all('button', class_ = 'see-proj-minutes')

[<button class="see-proj-minutes" data-home="0" data-nickname="Bulls" data-team="CHI">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="1" data-nickname="Pistons" data-team="DET">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="0" data-nickname="76ers" data-team="PHI">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="1" data-nickname="Heat" data-team="MIA">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="0" data-nickname="Wizards" data-team="WAS">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="1" data-nickname="Knicks" data-team="NYK">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="0" data-nickname="Pacers" data-team="IND">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="1" data-nickname="Raptors" data-team="TOR">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="0" data-nickname="Rockets" data-team

In [44]:
lineup_away = soup.find('ul', class_ = 'lineup__list is-visit')
starting_lineup_tags = lineup_away.find_all('li')[1:6]
[x.text.split('\n')[-2] for x in starting_lineup_tags]

['Josh Giddey', 'Coby White', 'Zach LaVine', 'P. Williams', 'N. Vucevic']

In [47]:
lineup_home = soup.find('ul', class_ = 'lineup__list is-home')
starting_lineup_tags = lineup_home.find_all('li')[1:6]
[x.text.split('\n')[-2] for x in starting_lineup_tags]

['C. Cunningham', 'Jaden Ivey', 'T. Hardaway', 'Tobias Harris', 'Jalen Duren']

In [48]:
[x for x in starting_lineup_tags.find_all('a', href = True)]

AttributeError: 'list' object has no attribute 'find_all'

In [50]:
[x['href'] for x in lineup_home.find_all('a', href = True)]

['/basketball/player/cade-cunningham-5336',
 '/basketball/player/jaden-ivey-5696',
 '/basketball/player/tim-hardaway-3459',
 '/basketball/player/tobias-harris-3211',
 '/basketball/player/jalen-duren-5703',
 '/basketball/player/tim-hardaway-3459',
 '/basketball/player/jaden-ivey-5696',
 '/basketball/player/bobi-klintman-5976',
 '/basketball/player/ausar-thompson-5932']

In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pretty_html_table import build_table
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(np.array([[0,1], [2, 3]]), columns = ['col1', 'col2'])

In [3]:
# Specify the email contents
mail = MIMEMultipart()
html = """\
<html><head></head><body>{0}</body></html>
""".format(build_table(df, 'grey_light', text_align = 'right', font_family = 'arial', width_dict = ['100','200','200','100','100','100','100'], font_size = 10))
mail.attach(MIMEText(html, 'html'))


In [4]:

# Set my email address and the password key
my_mail  = 'martinbog19@gmail.com'
# with open('gmail_key.txt') as f:
#     password = f.read()

# Set the subject of the email
mail['Subject'] = 'test_df'


In [ ]:

# Send the email (to myself)
server.sendmail(my_mail, my_mail, mail.as_string())
q = server.quit()

Email sent successfully!


In [9]:
import os

def is_running_in_github_actions():
    return os.getenv("GITHUB_ACTIONS") == "true"

In [12]:
os.getenv("k")

In [13]:
import pandas as pd
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import requests
import os
import json



if os.getenv("GITHUB_ACTIONS") == "true" :
  api_key = os.getenv('ODDS_API_KEY')
else :
  with open('secrets/odds_api_key.txt') as f:
    api_key = f.read()

with open('utils/odds_tm_map.json', 'r') as f :
  odds_tm_map = json.load(f)

games = pd.read_csv('data/games.csv')

# Store subset of games in the next 30 minutes
now = datetime.now(ZoneInfo('America/New_York'))
# games_now = games[(games['Time'] > now.strftime('%Y-%m-%d %H:%M:%S')) & (games['Time'] <= (now + timedelta(minutes = 30)).strftime('%Y-%m-%d %H:%M:%S'))]


games_now = games.tail(1)

# Iterate through games
for _, game in games_now.iterrows() :

    eventId = game['event_id']

    odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/basketball_nba/events/{eventId}/odds',
                             params = {'apiKey': api_key,
                                       'regions': 'us',
                                       'markets': 'player_first_basket',
                                       'oddsFormat': 'decimal'})
    

    bm_dfs = [pd.DataFrame(columns = ['name', 'price', 'bookmaker', 'update_time'])]
    for bookmaker in odds_response.json()['bookmakers'] :
        
        bm_df = pd.DataFrame(bookmaker['markets'][0]['outcomes'])
        bm_df['bookmaker'] = bookmaker['key']
        bm_df['update_time'] = bookmaker['markets'][0]['last_update']

        bm_dfs.append(
            bm_df
            .drop(columns = 'name')
            .rename(columns = {'description': 'name'})
        )

    game_df = pd.concat(bm_dfs).reset_index(drop = True)
    game_df['game_id'] = game['game_id']
    game_df['event_id'] = game['event_id']

    game_df['insert_timestamp_utc'] = datetime.now(timezone.utc)
    game_df.to_csv('data/odds_first_basket.csv', index = None, header = None, mode = 'a')

In [15]:
odds = pd.read_csv('data/odds_first_basket.csv')

In [21]:
odds[odds['game_id'] == '202412050GSW'].groupby('name').agg({'price': ['mean', 'min', 'max']})

price            
                         mean   min   max
name                                     
Alperen Sengun       6.375000   6.0   6.5
Andrew Wiggins       6.625000   6.5   7.0
Brandin Podziemski  10.875000   9.5  12.0
Buddy Hield          8.000000   8.0   8.0
Dillon Brooks        9.375000   9.0  10.5
Fred VanVleet        9.875000   9.5  10.0
Jabari Smith Jr     10.000000   9.5  11.0
Jalen Green          7.625000   7.0   8.0
Jonathan Kuminga     7.500000   6.5   8.5
Kevon Looney         8.625000   8.0  10.5
Moses Moody         12.333333  12.0  13.0